# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()    
except Exception as e:
    print(e)


#### Create Keyspace

In [6]:
# TO-DO: Create a Keyspace
try:
    session.execute(""" create keyspace if not exists project
    with replication = 
    {'class': 'SimpleStrategy', 'replication_factor': 1 }"""
)
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('project')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

To query this , will need to get the artist name, song name, and song length from the table, and will need to filter by sessionId and itemInSession

                    

The query would look like:

*select artist_name, song, length from music_app where sessionId = 338 and itemInSession = 4*

- Creating a table by the name music_app
- Primary key of the table consists of sessionId as partition key and ItemInSession as clustering key because we need to filter based on these attributes
- Table columns are: artist_name(text), song(text), length(float), sessionId(int), itemInSession(int)

In [9]:
#creating table
session.execute(""" create table if not exists music_app
(artist_name text, song text, length float, sessionId int, itemInSession int, primary key(sessionId, itemInSession)) """)

In [10]:
file = 'event_datafile_new.csv'

#inserting values into the table
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:
        query = "insert into music_app(artist_name, song, length, sessionId, itemInSession)"
        query = query + "values(%s,%s,%s,%s,%s)"
        session.execute(query, (line[0], line[9], float(line[5]), int(line[8]), int(line[3])))

#### Do a SELECT to verify that the data have been inserted into each table

In [11]:
rows = session.execute("""select artist_name, song, length from music_app where sessionId = 338 and itemInSession = 4
 """)

for row in rows:
    print(row.artist_name, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

## TO-DO Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

To query this, will need to get the artist name, song name, first name and last name of the user from the table and will need to filter by sessionId and userId and order by itemInSession

The query would look like:

*select itemInSession, artist_name, song, first_name, last_name from artist_user where userId = 10 and sessionId = 182*

- Creating a table by the name artist_user
- Primary key of the table consists of userId and sessionId as composite partition key and itemInSession as clustering key. It is so because we need to partition based on userId and sessionId as can be seen from the condition given in the query and itemInSession as clustering key because we need to order the results according to it. 
- Table columns are: artist_name(text), song(text), first_name(text), last_name(text), userId(int), sessionId(int), itemInSession(int)

In [12]:
#creating table
session.execute(""" create table if not exists artist_user
(artist_name text, song text, first_name text, last_name text, userId int, sessionId int, itemInSession int, primary key((userId , sessionId), itemInSession)) """)
                    

In [13]:
file = 'event_datafile_new.csv'

#inserting values into the table
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:
        query = "insert into artist_user(artist_name, song,first_name, last_name, userId, sessionId, itemInSession)"
        query = query + "values(%s,%s,%s,%s,%s,%s,%s)"
        session.execute(query, (line[0], line[9], line[1], line[4],int(line[10]), int(line[8]), int(line[3])))

In [14]:
rows = session.execute("""select itemInSession, artist_name, song, first_name, last_name from artist_user where userId = 10 and sessionId = 182""")

for row in rows:
    print(row.iteminsession, row.artist_name, row.song, row.first_name, row.last_name)

0 Down To The Bone Keep On Keepin' On Sylvie Cruz
1 Three Drives Greece 2000 Sylvie Cruz
2 Sebastien Tellier Kilometer Sylvie Cruz
3 Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


## TO-DO Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

To query this , will need to get the user first name and lastname from the table, and will need to filter by song name. There is a need to add the column userId to uniquely identify users.

### The query looks like:
*select firstName, lastName from users_history where song = 'All Hands Against His Own'*

- Creating a table by the name users_history
- Primary key of the table consists of song partition key and userId as clustering key. It is so because we need to partition based on song as can be seen from the condition given in the query and userId as clustering key because it helps to uniquely identify rows. 
- Table columns are: song(text), first_name(text), last_name(text), userId(int)

In [15]:
#creating table
session.execute("""create table if not exists users_history(
song text, firstName text, lastName text, userId int, primary key(song, userId)
)""")

In [16]:
file = 'event_datafile_new.csv'

#inserting values into the table
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "insert into users_history (song, firstName, lastName, userId)"
        query = query + " values (%s, %s, %s, %s)"
        session.execute(query, (line[9], line[1], line[4], int(line[10])))


In [17]:
rows = session.execute("""select firstName, lastName from users_history where song = 'All Hands Against His Own'""")

for row in rows:
    print(row.firstname, row.lastname)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Dropping tables

In [18]:
session.execute("""drop table music_app""")
session.execute("""drop table artist_user""")
session.execute("""drop table users_history""")

### Close the session and cluster connection¶

In [19]:
session.shutdown()
cluster.shutdown()